In [1]:
## #     3D split and Merge segmentation algorithm  #
#         Author = JY Ramel - June 2023          #
##################################################
# see the initial jupyter notebook if necessary 3Dsplitmerge.ipynb

# modifications applied but not working 

# Import the modules
import os
import time
import numpy as np
import nibabel as nib
import networkx as nx
from scipy.ndimage import median_filter,gaussian_filter
from itertools import combinations
import cv2
import argparse

#from matplotlib import pyplot as plt
#from scipy import ndimage as ndi
#import math
#from skimage import exposure


# Functions for pre-processing of the 3D image
def mg_filter_3d_nifti(input_data, sigma=0, kernel_size = (3,3,3)):
    """
    3D median or gaussian filtering : gaussian if sigma != 0 else median with kernel_size
    Parameters
    ----------
    input_data: the 3D volume
    (3,) ndarray
    kernel_size: the 3D size of the kernel
    [3,3,3)]
    sigma: for the gaussian filtering > 0
    1.5 float
    Returns
    -------
    the filtered volume        
    """    
    if sigma==0:
        # Apply 3D median filtering using scipy's median_filter
        filtered_data = median_filter(input_data, size=kernel_size)
        print("- Median filtering done.")
    else:
        # Apply true 3D smoothing using scipy's gaussian_filter
        filtered_data = gaussian_filter(input_data, sigma=sigma)
        print("- Gaussian filtering done.")
    
    return filtered_data.astype(np.uint16)



# Functions for gray level normalization of the 3D image
def normalize_data(data):
    """
    Normalize gray level to 255 stored in uint16 by using min and max values of the volume
    Parameters
    ----------
    input_data: the 3D volume
    Returns
    -------
    the normalized volume        
    """ 
    data_min = np.min(data)
    data_max = np.max(data)
    #normalized_data = (255*(data - data_min) / (data_max - data_min)).astype(np.uint16)
    normalized_data = (data - data_min) / (data_max - data_min)
    # Convert the normalized data to an array of integers in the range [0, 255]
    normalized_array = (normalized_data * 255).astype(np.uint16)

    print("- Gray level normalisation to 255 done.")
    return normalized_array


def augment_contrast(imgdata):
    """
    Augment contrast of the 3D volume by applying histogram equalization on each slice
    Parameters
    ----------
    imgdata: the 3D volume
    Returns
    -------
    the augmented volume
    """
    augmented_data = np.empty_like(imgdata)
    for i in range(imgdata.shape[-1]):
        augmented_data[..., i] = apply_histogram_equalization(imgdata[..., i])
    
    return augmented_data

def apply_histogram_equalization(image_slice):
    """
    Apply histogram equalization on a 2D image slice
    Parameters
    ----------
    image_slice: the 2D image slice
    Returns
    -------
    the augmented image slice
    """
    image_slice = cv2.equalizeHist(image_slice.astype(np.uint8))
    return image_slice


# The post-processing of a 3D segmentation to remove small regions
def remove_merge_again(labels, data_3Dimg, msize):
    """    
    Remove or merge small regions resulting from a segmentation (split and merge) and normalize label values to int16
    (number of ROI should be less than 32000)
    Parameters
    ----------
    labels: list of region labels
    data_3Dimg: the 3D volume containing the labels of the region for each voxel (0=background)
    msize: minimum size of a region
8   Returns
    -------
    final list of labels of regions
    the segmented 3D volume (with region_label on each voxel)
    """

    print("Post-processing to remove small regions (",msize,") and normalize label values to be saved in int16...")        
    data_img_copy = np.zeros_like(data_3Dimg, dtype=np.int16)
    newlabelvalue = 1
    fusion = 0

    for lab in labels :
        binary_mask = (data_3Dimg == lab)        
        segment_size = np.sum(binary_mask)
        if segment_size < msize or lab == 0:            
            data_img_copy[binary_mask == 1] = 0
            fusion+=1    # fusion with background - one  of the fusion is not a real one when lab == 0
        else:
            data_img_copy[binary_mask == 1] = newlabelvalue
            newlabelvalue = newlabelvalue+1
            
    print("Deletion / Fusion = ", fusion - 1)     
    new_labels = np.unique(data_img_copy)    

    return new_labels, data_img_copy


# 3D Crop function into a 3D volume / image 
def crop_3D(data, start_coord, end_coord):
    """
    Crop a volume inside the 3D data of a 3D image
    Parameters
    ----------
    data: the 3D volume
    (3,) ndarray
    The x,y,z minimum coordinates that delimite the crop
    xyzmax: (3,) ndarray
    The x,y,z maxymum coordinates that delimite the  crop
    Returns
    -------
    the cropped volume        
    """
    # Convert coordinates to integers
    start_coord = np.round(start_coord).astype(int)
    end_coord = np.round(end_coord).astype(int)

    # Crop the subcube
    cropped_data = data[start_coord[0]:end_coord[0], start_coord[1]:end_coord[1], start_coord[2]:end_coord[2]]

    return cropped_data



# Create Object Cube as a dictionary 
def create_cube(
        xyzmin,
        xyzmax,
        graymin,
        graymax,
        graymean,
        variance,
        leaf=False,
        level=0):
    """
    Create a cube object with appropriate descriptors coming from the 3D image
    Parameters
    ----------
    xyzmin: the x,y,z minimum coordinates of the cube
    xyzmax: the x,y,z maximum coordinates of the cube
    graymin: the minimum gray level inside the cube
    graymax: the maximum gray level inside the cube
    graymean: the mean gray level inside the cube
    variance: the variance of gray level inside the cube
    leaf: boolean to say if the cube is a leaf or not
    level: the level of the cube in the octree
    Returns
    -------
    a cube object with attributes : ndarray
    """
    volume = (xyzmax[0]-xyzmin[0])*(xyzmax[1]-xyzmin[1])*(xyzmax[2]-xyzmin[2])
    cube = {
        'xyzmin':np.array(xyzmin, dtype = int), 
        'xyzmax':np.array(xyzmax, dtype = int), 
        'leaf':leaf,
        'level':level,
        'graymin':graymin,
        'graymax':graymax,
        'graymean':graymean,
        'variance':variance,
        'volume':volume,
        'region_label':0
        }    
    return cube


# A function to sort the edges of a graph accoring an attribute
def sort_edges_by_attribute(graph, attribute):
    sorted_edges = sorted(graph.edges(data=True), key=lambda x: x[2].get(attribute, float('inf')))
    return sorted_edges


def find_cube_edges(su_x, su_y, su_z):
    """
    Find the points of the edges of a cube
    Parameters
    ----------
    su_x: the x minimum and maximum coordinates of the cube
    su_y: the y minimum and maximum coordinates of the cube
    su_z: the z minimum and maximum coordinates of the cube
    Returns
    -------
    a list of the points of the edges of the cube
    """
    arrete_points = []
    [arrete_points.append((x, su_y[0], su_z[0])) for x in range(su_x[0], su_x[1]+1)]
    [arrete_points.append((x, su_y[0], su_z[1])) for x in range(su_x[0], su_x[1]+1)]
    [arrete_points.append((x, su_y[1], su_z[0])) for x in range(su_x[0], su_x[1]+1)]
    [arrete_points.append((x, su_y[1], su_z[1])) for x in range(su_x[0], su_x[1]+1)]
    [arrete_points.append((su_x[0], y, su_z[0])) for y in range(su_y[0], su_y[1]+1)]
    [arrete_points.append((su_x[0], y, su_z[1])) for y in range(su_y[0], su_y[1]+1)]
    [arrete_points.append((su_x[1], y, su_z[0])) for y in range(su_y[0], su_y[1]+1)]
    [arrete_points.append((su_x[1], y, su_z[1])) for y in range(su_y[0], su_y[1]+1)]
    [arrete_points.append((su_x[0], su_y[0], z)) for z in range(su_z[0], su_z[1]+1)]
    [arrete_points.append((su_x[0], su_y[1], z)) for z in range(su_z[0], su_z[1]+1)]
    [arrete_points.append((su_x[1], su_y[0], z)) for z in range(su_z[0], su_z[1]+1)]
    [arrete_points.append((su_x[1], su_y[1], z)) for z in range(su_z[0], su_z[1]+1)]
    return arrete_points


# A function to find the adjacencies of all cubes in the graph
def find_cube_adjacencies(g):
    """
    Find the adjacencies of all cubes in the graph
    Parameters
    ----------
    g: the graph corresponding to the splitted 3D volume
    Returns
    -------
    the number of edges in the graph
    """
    print("Creating matrix")
    N = g.number_of_nodes() + 1
    nbedges = 0
    matrice_3D = {}

    for s in range(1, N):
        su_x = g.nodes[s]['xyzmin'][0], g.nodes[s]['xyzmax'][0]
        su_y = g.nodes[s]['xyzmin'][1], g.nodes[s]['xyzmax'][1]
        su_z = g.nodes[s]['xyzmin'][2], g.nodes[s]['xyzmax'][2]

        arrete_points = find_cube_edges(su_x, su_y, su_z)
        
        for (x, y, z) in arrete_points:
            if (x, y, z) not in matrice_3D:
                matrice_3D[(x, y, z)] = []
            if s not in matrice_3D[(x, y, z)]:
                matrice_3D[(x, y, z)].append(s)

    for _, adj_nodes in matrice_3D.items():
        couples = combinations(adj_nodes, 2)
        for couple in couples:
            grmin = min(g.nodes[couple[0]]['graymin'], g.nodes[couple[1]]['graymin'])
            grmax = max(g.nodes[couple[0]]['graymax'], g.nodes[couple[1]]['graymax'])
            toplevel = min(g.nodes[couple[0]]['level'], g.nodes[couple[1]]['level'])
            attributs = {'level': toplevel, 'graymin': grmin, 'graymax': grmax}
            if not g.has_edge(couple[0], couple[1]):
                g.add_edge(couple[0], couple[1], **attributs)
                nbedges += 1

    return nbedges



# split a cube into 8 sub-parts to expand the octree
def split(cube, minsize=4):
    """
    Splits the cube into 8 sub-nodes
    Parameters
    ----------
    cube: dictionary
    
    minsize:  int
    min size of the chiild cubes (less than 4 can raise pb when cropping - see also cutting boundaries below)

    Returns
    -------
    octree : (8,2,3) ndarray
        return an array of 8 sub-cubes
        The x,y,z minimum and maximum coordiantes of the sub-cubes are grouped along the second dimension
    """

    xyzmin = cube["xyzmin"]
    xyzmax = cube["xyzmax"]
    
    if(xyzmax[0]-xyzmin[0] < minsize or xyzmax[1]-xyzmin[1] < minsize or xyzmax[2]-xyzmin[2] < minsize) :
        octree=  np.zeros([8,2,3], dtype = float )                
    else:
        xyzmed = (xyzmax + xyzmin) / 2
    
        octree = np.array([
        [[xyzmin[0], xyzmin[1], xyzmin[2]], [xyzmed[0], xyzmed[1], xyzmed[2]]],
        [[xyzmin[0], xyzmed[1], xyzmin[2]], [xyzmed[0], xyzmax[1], xyzmed[2]]],
        [[xyzmed[0], xyzmed[1], xyzmin[2]], [xyzmax[0], xyzmax[1], xyzmed[2]]],
        [[xyzmed[0], xyzmin[1], xyzmin[2]], [xyzmax[0], xyzmed[1], xyzmed[2]]],
        [[xyzmin[0], xyzmin[1], xyzmed[2]], [xyzmed[0], xyzmed[1], xyzmax[2]]],
        [[xyzmin[0], xyzmed[1], xyzmed[2]], [xyzmed[0], xyzmax[1], xyzmax[2]]],
        [[xyzmed[0], xyzmed[1], xyzmed[2]], [xyzmax[0], xyzmax[1], xyzmax[2]]],
        [[xyzmed[0], xyzmin[1], xyzmed[2]], [xyzmax[0], xyzmed[1], xyzmax[2]]], 
        ]) 
      
    return octree
    


# convert the octree (8 childs of a cube) into leafs or cubes according to the 3D image info
def octree2cube(cube, octree, imgdata, threshold, strategy):
    """
    Converts simple cube with xoord into cube objects with appropriate descriptors coming from the 3D image
    Parameters
    ----------
    cube: the parent cube
    octree:  the 8 children cubest
    threshold : similarity value for voxel intensity inside a cube to decide to split it again (leaf == false) or not (leaf == true)
    Returns
    -------
    a list of 8 cubes with attributes : ndarray        
    """

    clist = []
    newlevel = cube['level'] + 1
    
    for o in octree :
        xyzmin = o[0].astype(int)
        xyzmax = o[1].astype(int)
        cropped_data = crop_3D(imgdata, xyzmin,xyzmax)
        gmin = np.min(cropped_data)
        gmax = np.max(cropped_data)
        gmean = np.mean(cropped_data)
        variance = np.var(cropped_data)
        #if (gmax-gmin < threshold):
        if (strategy == "minMax" and gmax - gmin < threshold) or (strategy == "variance" and np.sqrt(variance) < threshold):
            newc = create_cube(xyzmin, xyzmax, gmin, gmax, gmean, variance, leaf=True, level=newlevel)
        else:
            newc = create_cube(xyzmin, xyzmax, gmin, gmax, gmean, variance, leaf=False, level=newlevel)
        clist.append(newc)
    return clist
    


# The SPLIT loop...
def split3D(data_3Dimg, rootc, min_size, sim_threshold, strategy):
    """
    Split a 3D img in octree according gray level values
    Parameters
    ----------
    data-3Dimg: the 3D volume of the 3D image to split
    rootc:  the root cube corresponding to the image size
    min_size : minimun size of a cube during the split
    sim_threshold: gray level homogeneity threshold to stop splitting
    Returns
    -------
    a graph resulting of the split. Each node is a cube with attributes and each edge is an adjacency between 2 cubes
    the segmented volume
    """
    cpt = 0               # used just to print feedbacks regularely

    cubestoprocess = []
    leafGraph = nx.Graph()
    data_img_copy = np.copy(data_3Dimg)

    cubestoprocess.append(rootc)

    while len(cubestoprocess) > 0 :    
        cub = cubestoprocess.pop()    
        oct = split(cub, minsize=min_size)
        if np.any(oct) :
            childlist = octree2cube(cub, oct, data_3Dimg, sim_threshold, strategy)
            for elmt in childlist:

                if elmt['leaf'] == True :
                    #add to graph
                    # Add the leaf node in the RAG / before it was done with add_leaf_node(leafGraph,elmt)
                    id = leafGraph.number_of_nodes()+1
                    leafGraph.add_node(id)
                    nx.set_node_attributes(leafGraph, {id:elmt})            

                    data_img_copy[ elmt['xyzmin'][0]: elmt['xyzmax'][0], elmt['xyzmin'][1]: elmt['xyzmax'][1], elmt['xyzmin'][2]: elmt['xyzmax'][2]] = int( (elmt['graymax'] + elmt['graymin'])/2 )
                    #print("- Level = ", elmt['level']," Similarity = ", elmt['graymax'] - elmt['graymin'] )
                else:
                    #add to process
                    cubestoprocess.append(elmt)        
        else:
            #cub is Too small to be splitted so become a leaf  
            cub['leaf'] = True
            #Add the leaf node in the RAG / before it was done with add_leaf_node(leafGraph,cub)
            id = leafGraph.number_of_nodes()+1
            leafGraph.add_node(id)
            nx.set_node_attributes(leafGraph, {id:cub})            
            data_img_copy[ cub['xyzmin'][0]: cub['xyzmax'][0], cub['xyzmin'][1]: cub['xyzmax'][1], cub['xyzmin'][2]: cub['xyzmax'][2]] = int( (cub['graymax'] + cub['graymin'])/2 )
            
        # Just to provide feedbacks of the progress...
        cpt+=1
        if cpt > 50 :
            print("- Queue : ",len(cubestoprocess)," - Level = ", elmt['level']," Similarity = ", elmt['graymax'] - elmt['graymin'] )                                
            cpt = 0

    print("Split done.")
    return leafGraph, data_img_copy
    


# The MERGE loop....
def merge3D(data_3Dimg, g, sim_threshold, strategy):
    """    
    Merge splitted regions resulting of a split in octree occording to a RAG graph and to gray levels inside region 
    Parameters
    ----------
    data_3Dimg: the 3D volume of the 3D image to split
    g: the graph corresponding to the splitted 3D volume 
    sim_threshold: gray level homogeneity threshold to stop splitting
    Returns
    -------
    final number of regions
    the segmented 3D volume (with region_label on each voxel)
    """

    #data_img_copy = np.copy(data_3Dimg)
    data_img_copy = np.zeros_like(data_3Dimg, dtype=np.uint16)

    # Give a label to each Region 
    regionid = 1
    for node in g.nodes:                      
        # Background should be associated with a specific processing 
        if g.nodes[node]['graymax'] > 0 :
            # Not Background
            g.nodes[node]['region_label'] = regionid
            regionid+=1
        else:
            # Is Background
            g.nodes[node]['region_label'] = 0

        # g.nodes[node]['region_label'] = regionid
        # regionid+=1       

        # Initialize the label image with each node label
        data_img_copy[ g.nodes[node]['xyzmin'][0]: g.nodes[node]['xyzmax'][0], g.nodes[node]['xyzmin'][1]: g.nodes[node]['xyzmax'][1], g.nodes[node]['xyzmin'][2]: g.nodes[node]['xyzmax'][2]] = (g.nodes[node]['region_label'])             
        # end of loop on nodes

    print("Nb of ROI before merge :",regionid)

    #Label propagation between neighbors
    sorted_edges = sort_edges_by_attribute(g, 'level')    # Start from big regions
    fusion = nofusion = 0

    for edge in sorted_edges:        
        node1, node2, attributes = edge                       
        #there are different possibilities to fill gmin and gmax below... 
        gmax = attributes['graymax'] # could be max with gmax values of the 2 nodes because these values change along merging 
        gmin = attributes['graymin'] # could be min with gmin values of the 2 nodes
        vol1 = g.nodes[node1]['volume']
        vol2 = g.nodes[node2]['volume']
        mean1 = g.nodes[node1]['graymean']
        mean2 = g.nodes[node2]['graymean']
        var1 = g.nodes[node1]['variance']
        var2 = g.nodes[node2]['variance']
        vol_global = vol1 + vol2
        gray_mean_global = (vol1 * mean1 + vol2 * mean2) / vol_global
        var_globale = vol1 * (var1 + (gray_mean_global - mean1)**2) + vol2 * (var2 + (gray_mean_global - mean2)**2) / (vol_global)
        
        # if gmax - gmin < sim_threshold:
        # if np.sqrt(var_globale) < sim_threshold:
        if (strategy == "minMax" and gmax - gmin < sim_threshold) or (strategy == "variance" and np.sqrt(var_globale) < sim_threshold):      
            # Update label image
            data_img_copy[ g.nodes[node1]['xyzmin'][0]: g.nodes[node1]['xyzmax'][0], g.nodes[node1]['xyzmin'][1]: g.nodes[node1]['xyzmax'][1], g.nodes[node1]['xyzmin'][2]: g.nodes[node1]['xyzmax'][2]] = (g.nodes[node1]['region_label']) 
            data_img_copy[ g.nodes[node2]['xyzmin'][0]: g.nodes[node2]['xyzmax'][0], g.nodes[node2]['xyzmin'][1]: g.nodes[node2]['xyzmax'][1], g.nodes[node2]['xyzmin'][2]: g.nodes[node2]['xyzmax'][2]] = (g.nodes[node1]['region_label']) 
            
            # upDate nodes attributes
            g.nodes[node2]['graymax'] = g.nodes[node1]['graymax'] = gmax
            g.nodes[node2]['graymin'] = g.nodes[node1]['graymin'] = gmin
            g.nodes[node2]['variance'] = g.nodes[node1]['variance'] = var_globale
            g.nodes[node2]['graymean'] = g.nodes[node1]['graymean'] = gray_mean_global
            g.nodes[node2]['volume'] = g.nodes[node1]['volume'] = vol_global
            g.nodes[node2]['region_label'] = g.nodes[node1]['region_label']
            fusion+=1                                          
        else:
            #update only label image
            data_img_copy[ g.nodes[node1]['xyzmin'][0]: g.nodes[node1]['xyzmax'][0], g.nodes[node1]['xyzmin'][1]: g.nodes[node1]['xyzmax'][1], g.nodes[node1]['xyzmin'][2]: g.nodes[node1]['xyzmax'][2]] = (g.nodes[node1]['region_label'])
            data_img_copy[ g.nodes[node2]['xyzmin'][0]: g.nodes[node2]['xyzmax'][0], g.nodes[node2]['xyzmin'][1]: g.nodes[node2]['xyzmax'][1], g.nodes[node2]['xyzmin'][2]: g.nodes[node2]['xyzmax'][2]] = (g.nodes[node2]['region_label']) 
            nofusion+=1                               

    print("Fusion=",fusion," - nofusion=",nofusion)        
    labels = np.unique(data_img_copy)    

    return labels, data_img_copy



# the split and merge function
def split_merge_3D(T_split, T_merge, minimum_size, strategy, data, rootc, nifti_image, output_path=""):
    """    
    Split and Merge a 3D nifti image with gray value intensities corresponding to an octree starting from rootc volume
    Parameters
    ----------
    T: Threshold of similarity of intensities / gray level values
    F: minimum lenght of a side for the splitted cubes
    data: the 3D volume 
    rootc: the  cube corresponding to the shape of the image / 3D volume
    nifti_image: the corresponding nifti image (needed just to save partial result of the split step in a nifti file)
    output_path: the path where to save the partial result (optional)
    Returns
    -------
    array of labels given to the regions
    the segmented 3D volume (with region_label on each voxel)
    """

    # Split3D function
    print("Splitting (+ edges) takes time ...")
    leafGraph, seg_data = split3D(data, rootc, minimum_size, T_split, strategy)
    print("G size: |V|=",leafGraph.number_of_nodes(), "|E|=",leafGraph.number_of_edges())
    
    print("RAG creation (can takes a lot of time ...)")
    find_cube_adjacencies(leafGraph)
    print("RAG size: |V|=",leafGraph.number_of_nodes(), "|E|=",leafGraph.number_of_edges())
    
    # Save split result in a niftii file if provided
    # if output_path != "":
    #     splitted_nifti = nib.Nifti1Image(seg_data, affine=nifti_image.affine, header=nifti_image.header)
    #     nib.save(splitted_nifti, output_path)

    # Merge3D function
    print("Merging...")
    llabels, final_data = merge3D(seg_data, leafGraph, T_merge, strategy)
    
    return llabels, final_data



#generate colors
def generate_basic_colors(N):
    basic_colors = [
        (255, 0, 0),      # Red
        (0, 255, 0),      # Green
        (0, 0, 255),      # Blue
        (255, 255, 0),    # Yellow
        (255, 0, 255),    # Magenta
        (0, 255, 255),    # Cyan
        (128, 128, 128),  # Gray
        (128, 0, 0),      # Maroon
        (0, 128, 0),      # Olive
        (0, 0, 128),      # Navy
        (128, 128, 0),    # Olive Green
        (128, 0, 128),    # Purple
        (0, 128, 128),    # Teal
        (128, 128, 192),  # Sky Blue
        (128, 64, 64),    # Brown
        (64, 128, 64),    # Dark Green
        (64, 64, 128),    # Indigo
        (128, 128, 64),   # Yellow Green
        (128, 64, 128),   # Dark Purple
        (64, 128, 128)    # Light Teal
    ]
    
    N = (N + 1) % len(basic_colors)
    return basic_colors[N]
    

# Create an ITK label file with random color for the list of found ROIs
def create_itk_labels_file(labels, filename, comment=""):
    """
    Create an ITK label file with random color for the list of found ROIs
    Parameters
    ----------
    labels: list of labels
    filename: name of the file to create
    comment: comment to add at the beginning of the file
    Returns
    -------
    Nothing
    """
    cpt = 0
    # Open the log file in append mode
    with open(filename, 'w') as f:
        f.write("### Label file in ITK Snap format for " + filename + "\r\n")
        f.write(comment + "\r\n\r\n")

        # Loop through the labels 
        for label in labels :
            # Write the information              
            if label == 0:
                r,v,b = (0,0,0)
            else:
                r,v,b = generate_basic_colors(cpt)                   
            tline = " " + str(int(label)) + " " + str(r) + " " + str(v) + " " + str(b) + " \"ROI " + str(int(label)) + "\" \r\n"                                   
            f.write(tline)
            cpt += 1            
    f.close()

    



###############################################
# Loop to segment all the images inside a directory
def split_merge_3D_batch(input_dir, output_dir, T = 20, F = 9):
    """    
    Loop to segment all the images inside a directory
    Parameters
    ----------
    input_dir: directory containing the images to process (nii.gz)
    output_dir : directory to put the results (should exist)
    # SPLI & MERGE PARAMETERS
    T: gray level homogeneity threshold
    F: Minimun size during split
    Returns
    -------
    Nothing
    """
     
    print("Running batch.... ")  
    
    # Open the log file in append mode
    log_file = output_dir + 'C:/Users/cassa/Documents/INFO S4 L2/splitandmerge3D (2)/cailles/c_matrix_AB.txt'

    f = open(log_file, 'w')
    
    # Enumerate NIfTI files in the input directory
    nifti_files = [file for file in os.listdir(input_dir) if file.find("label") == -1 and file.endswith('.nii.gz')]

    # Loop through each NIfTI file
    for nifti_file in nifti_files:

        start_time = time.time()

        # Construct the file paths
        input_path = os.path.join(input_dir, nifti_file)
        output_path = os.path.join(output_dir, nifti_file)    
        
        info = split_merge_3D_1img(input_path,output_path)

        # Update log file with processed image info
        f.write(f"Processed image : {output_path}")  
        f.write(info)  
        f.write(f" - processing time = {(time.time() - start_time)}\n")            
        f.write("\n")
        f.flush()

        print(f"Image {output_path} saved. \n\n")
        
    print("Batch done")
    f.close()




###############################################
# THE MAIN SPLIT AND MERGE FUNCTION 
def split_merge_3D_1img(image_path,output_path, T_split, T_merge, minimum_size, strategy):
    """
    The main split and merge 3D function for one image including preprocessing and post processing.
    -------
    Parameters
    -------
    image_path:  image to process (nii.gz)
    output_path : output path of the resulting segmented image (path of the itk label file decuded from this one)
    REM: for the moment the SPLIT & MERGE PARAMETERS have to be set manually in this function
    T_split: gray level homogeneity threshold during split
    T_merge: gray level homogeneity threshold during merge
    minimum_size: used by post processing to remove small region returned by split and merge (actually set to 2xFxFxF)
    -------
    Returns
    -------
    String containing info about the result
    """

    output_path_label = output_path  + ".label"
        
    # Load NIfTI image
    nifti_file = image_path
    nifti_image = nib.load(nifti_file)
    imgdata = nifti_image.get_fdata()
    origin = nib.Nifti1Image(imgdata, affine=nifti_image.affine)
    nib.save(origin, 'caille\\caille_origin.nii.gz')
    gmin = np.min(imgdata)
    gmax = np.max(imgdata)
    gmean = np.mean(imgdata)
    print("Initial Image: shape = ",imgdata.shape, " - (min,max)=",gmin,gmax)
    
    # Pre-processing on the image     
    print("Pre-processing image...")
    data = augment_contrast(imgdata)
    data = normalize_data(data)          # mandatory step
    data = mg_filter_3d_nifti(data, sigma=0.5, kernel_size = (3,3,3))
    # data = mg_filter_3d_nifti(imgdata, sigma=1.5)
    prepro = nib.Nifti1Image(data, affine=nifti_image.affine)
    nib.save(prepro, 'caille\\caille_prepro.nii.gz')


    # Corresponding Root Cube creation
    rootc = create_cube([0,0,0],data.shape, np.min(data), np.max(data), np.mean(data), np.var(data))
    print("Initial Cube=", rootc, " - data type=",data.dtype )
    print("- Split & Merge 3D with T=",T_split," - Minimum size of a cube=",minimum_size)
    rlabels, result_data = split_merge_3D(T_split, T_merge, minimum_size, strategy, data, rootc, nifti_image)
    print("- Nb of Regions = ", len(rlabels)," - Labels = ", rlabels , " - data type = ",result_data.dtype)

    # Post-processing to remove some regions ???
    minimum_size = 2*minimum_size*minimum_size*minimum_size     #minimum volume = k x (volume of the minimum cube)
    # rlabels, result_data = remove_merge_again(rlabels, result_data, minimum_size)

    # print("- Final nb of Regions (in ITK file) = ", len(rlabels)," - Labels = ", rlabels, " - data type = ",result_data.dtype)


    # Save the final segmentation result in a niftii file
    #final_nifti = nib.Nifti1Image(final_data, affine=nifti_image.affine, header=nifti_image.header)
    final_nifti = nib.Nifti1Image(result_data, affine=nifti_image.affine)
    print(output_path)
    nib.save(final_nifti, output_path)

    # Create the labels file for compatibility with 3DBrainMiner
    commentaire = "### Split & Merge 3D with T_split=" + str(T_split) + " - T_merge=" + str(T_merge) + " - F=" + str(minimum_size) + " - min-Size="  + str(minimum_size) + " after 3D medianfilter - Nb of ROI=" + str(len(rlabels)) + " (" + str(len(rlabels)) + ")"
    create_itk_labels_file(rlabels, output_path_label,commentaire)
    print(commentaire)

    # Save all Region masks into niftii files
    #for lbl in llabels :
    #    output_p = "C:\\DATA\\BD_Images\\INRAE\\Zenodo\\zenodo-dataset\\MRI\\RESULTS\\RESULT_" + str(lbl) + "_IMG.nii.gz"
    #    create_mask_niftii(lbl,final_data, output_p,nifti_image)
    #print("Binary masks Saved")

    return commentaire



##############################################################
# THE MAIN()


### Argument parser ###
def build_arg_parser() -> argparse.ArgumentParser:

    example_text = '''example: python10.exe splitandmerge3D.py --input-file "myBrain/MyBrain.nii.gz" --output-folder "myBrain/segmentation" --split-threshold 15 --merge-threshold 41 --min-size 6 '''

    parser = argparse.ArgumentParser(
        description="Segment a 3D brain image using the Split & Merge algorithm",
        epilog=example_text,
    )
    parser.add_argument(
        "--input-file", type=str, help="Input imagery path (NiFTI file)", required=True
    )
    parser.add_argument(
        "--output-folder", type=str, help="Output folder path, the name of the file will be '{folder name}_seg_Tsplit-{split threshold}_Tmerge-{merge threshold}_minSize-{minimum size}.nii.gz'",
        default="segmentation",
    )
    parser.add_argument(
        "--merge-threshold",
        type=int,
        help="threshold for merging regions (default: 40)",
        default=40,
    )
    parser.add_argument(
        "--split-threshold",
        type=int,
        help="threshold for splitting regions (default: 40)",
        default=40,
    )
    parser.add_argument(
        "--min-size",
        type=int,
        help="minimum size of a region (default: 10)",
        default=10,
    )
    parser.add_argument(
        "--homogeneity-strategy",
        type=str,
        choices=["minMax", "variance"],
        help="choose strategy for calculating the homogeneousity of a region. (default: minMax)",
        default=1,
    )
    return parser


if __name__ == "__main__":
    start_time = time.time()
    arg_parser = build_arg_parser()
    args = arg_parser.parse_args()
    output = args.output_folder + f"/seg_Tsplit-{args.split_threshold}_Tmerge-{args.merge_threshold}_minSize-{args.min_size}_strat-{args.homogeneity_strategy}.nii.gz"
    print(f"Intput MRI path : {args.input_file}")
    print(f"Output path : {output}")
    print(f"Merge threshold : {args.merge_threshold}")
    print(f"Split threshold : {args.split_threshold}")
    print(f"Minimum size : {args.min_size}")
    print(f"Strategy : {args.homogeneity_strategy}")

    split_merge_3D_1img(args.input_file, output, args.split_threshold, args.merge_threshold, args.min_size, args.homogeneity_strategy)

    print("--- %s seconds ---" % (time.time() - start_time))
    


C:\Users\cassa\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
usage: ipykernel_launcher.py [-h] --input-file INPUT_FILE [--output-folder OUTPUT_FOLDER]
                             [--merge-threshold MERGE_THRESHOLD] [--split-threshold SPLIT_THRESHOLD]
                             [--min-size MIN_SIZE] [--homogeneity-strategy {minMax,variance}]
ipykernel_launcher.py: error: the following arguments are required: --input-file


SystemExit: 2

C:\Users\cassa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
def calcul_volume(coordinates):

    def __init__(self,indice, coordXmax,coordYmax,coordZmax, coordXmin, coordYmin, coordZmin, volume, idNoeud) -> None:
        
        self.indice = indice         # indice du noeud
        self.coordX = coordXmax      # coordX maximal de la région
        self.coordY = coordYmax        
        self.coordZ = coordZmax
        self.volume = volume
        self.coordXmin = coordXmin   # coordY minimal de la région
        self.coordYmin= coordYmin
        self.coordZmin= coordZmin
        
        graph={}
        L=[i for i in regionid]
        liste_noeuds=[random.choice(string.ascii_lowercase) for i in range(len(L))]
        for i in range(len(liste_noeuds)):
            if i<len(L):
                graph[liste_noeuds[i]]=L[i]
            else:
                print("Pas assez de chiffres pour tous les noeuds.")
        print("Graphe associant les noeuds aux chiffres :", graph)2..
            
        
        